# Active flow control of a 2D circular cylinder
This is a example notebook for DRL-based active flow control in a 2D cylinder.
For more details, see the paper [DRLinFluids: An open-source Python platform of coupling deep reinforcement learning and OpenFOAM](https://aip.scitation.org/doi/10.1063/5.0103113).

## Step 1: Import the necessary modules and define the OpenFOAM environment class from the `drlinfluids.environments` module.

For the definition of member functions, please refer to [Tensorforce documentation](https://tensorforce.readthedocs.io/en/latest/environments/environment.html).

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from time import time

from drlinfluids.environments.tensorforce import OpenFoam
from drlinfluids import utils as utils
import drlinfluids

class FlowAroundCylinder2D(OpenFoam):
    def states(self):
        if self.state_params['type'] == 'pressure':
            return dict(
                type='float',
                shape=(int(self.state_params['probe_info'].shape[0]), )
            )

        elif self.state_params['type'] == 'velocity':
            if self.foam_params['num_dimension'] == 2:
                return dict(
                    type='float',
                    shape=(int(2 * self.state_params['probe_info'].shape[0]), )
                )
            elif self.foam_params['num_dimension'] == 3:
                return dict(
                    type='float',
                    shape=(int(3 * self.state_params['probe_info'].shape[0]), )
                )
            else:
                assert 0, 'Simulation type error!'

        else:
            assert 0, 'No define state type error!'

    def actions(self):
        return dict(
            type='float',
            shape=(1, ),
            min_value=self.agent_params['minmax_value'][0],
            max_value=self.agent_params['minmax_value'][1]
        )

    def agent_actions_decorator(self, actions):
        if self.num_trajectory < 1.5:
            new_action = 0.4 * actions
        else:
            new_action = np.array(self.decorated_actions_sequence[self.num_trajectory - 2]) \
                         + 0.4 * (np.array(actions) - np.array(self.decorated_actions_sequence[self.num_trajectory - 2]))
        return new_action

    
    def execute(self, actions=None):
        self.trajectory_start_time = time()
        self.num_trajectory += 1
        if actions is None:
            print("carefully, no action given!")

        self.decorated_actions = self.agent_actions_decorator(actions)

        self.actions_sequence = np.append(self.actions_sequence, actions)

        if self.num_trajectory < 1.5  :
            self.start_actions=[self.actions_sequence[0]]
            self.end_actions = [self.actions_sequence[0]]
        else:
            self.start_actions=[self.actions_sequence[-2]]
            self.end_actions=[self.actions_sequence[-1]]

        start_time_float = np.around(
            float(self.cfd_init_time) + (self.num_trajectory - 1) * self.agent_params['interaction_period'],
            decimals=self.decimal
        )
        end_time_float = np.around(start_time_float + self.agent_params['interaction_period'], decimals=self.decimal)

        self.start_time_filename, self.start_time_path = utils.get_current_time_path(self.foam_root_path)

        utils.dict2foam(
            self.start_time_path,
            utils.actions2dict(self.agent_params['entry_dict_q0'], self.agent_params['variables_q0'], self.start_actions)
        )

        utils.dict2foam(
            self.start_time_path,
            utils.actions2dict(self.agent_params['entry_dict_q1'], self.agent_params['variables_q1'], self.end_actions)
        )

        start_time=[start_time_float]
        utils.dict2foam(
            self.start_time_path,
            utils.actions2dict(self.agent_params['entry_dict_t0'], self.agent_params['variables_t0'], start_time)
        )

        
        simulation_start_time = time()
        drlinfluids.runner.run(
            self.foam_root_path,
            self.foam_params, self.agent_params['interaction_period'], self.agent_params['purgeWrite_numbers'],self.agent_params['writeInterval'],
            self.agent_params['deltaT'],
            start_time_float, end_time_float
             )
        simulation_end_time = time()

        self.probe_velocity = utils.read_foam_file(
            self.foam_root_path + f'/postProcessing/probes/{self.start_time_filename}/U',
            dimension=self.foam_params['num_dimension']
        )

        self.probe_pressure = utils.read_foam_file(
            self.foam_root_path + f'/postProcessing/probes/{self.start_time_filename}/p',
            dimension=self.foam_params['num_dimension']
        )

        self.force = utils.resultant_force(
            utils.read_foam_file(
            self.foam_root_path + f'/postProcessing/forcesIncompressible/{self.start_time_filename}/forces.dat'
            )
        )

        self.force_Coeffs = utils.read_foam_file(
            self.foam_root_path + f'/postProcessing/forceCoeffsIncompressible/{self.start_time_filename}/forceCoeffs.dat'
        )

        if self.num_trajectory < 1.5:
            self.history_force = self.force
            self.history_force_Coeffs = self.force_Coeffs
        else:
            self.history_force = pd.concat([self.history_force, self.force[1:]]).reset_index(drop=True)
            self.history_force_Coeffs = pd.concat(
                [self.history_force_Coeffs, self.force_Coeffs[1:]]
            ).reset_index(drop=True)


        if self.state_params['type'] == 'pressure':
            next_state = self.probe_pressure.iloc[-1, 1:].to_numpy()
        elif self.state_params['type'] == 'velocity':
            next_state = self.probe_velocity.iloc[-1, 1:].to_numpy()
        else:
            next_state = False
            assert next_state, 'No define state type'

        self.state_data = np.append(self.state_data, next_state)

        self.decorated_actions_sequence = np.append(self.decorated_actions_sequence,  actions)

        reward = self.reward_function()
        self.trajectory_reward = np.append(self.trajectory_reward, reward)
        self.episode_reward += reward
        print(self.num_trajectory,self.start_actions,self.end_actions,reward)

        terminal = False

        self.trajectory_end_time = time()

        self.exec_info = {
            'episode': self.num_episode,
            'trajectory': self.num_trajectory,
            'start_time_float': start_time_float,
            'end_time_float': end_time_float,
            'timestampStart': self.trajectory_start_time,
            'timestampEnd': self.trajectory_end_time,
            'current_trajectory_reward': reward,
            'episode_reward': self.episode_reward,
            'actions': actions,
            'cfd_running_time': simulation_end_time - simulation_start_time,
            'number_cfd_timestep': int(np.around((end_time_float - start_time_float) / self.foam_params['delta_t'])),
            'envName': self.foam_root_path.split('/')[-1],  #str
            'current_state': self.state_data[-2],
            'next_state': next_state
        }
        self.info_list.append(self.exec_info)
        
        return next_state, terminal, reward

    def reward_function(self):
        vortex_shedding_period = 0.025
        drug_coeffs_sliding_average = drlinfluids.utils.force_coeffs_sliding_average(self.history_force_Coeffs, vortex_shedding_period, self.foam_params['delta_t'])[0]
        lift_coeffs_sliding_average = self.force_coeffs_sliding_average(self.history_force_Coeffs, vortex_shedding_period, self.foam_params['delta_t'])[1]
        return 3.205 - drug_coeffs_sliding_average - 0.1 * np.abs(lift_coeffs_sliding_average)


    def force_coeffs_sliding_average(self, sliding_time_interval):
        sampling_num = int(sliding_time_interval / self.foam_params['delta_t'])
        if self.history_force_Coeffs.shape[0] <= sampling_num:
            sliding_average_cd = np.mean(signal.savgol_filter(self.history_force_Coeffs.iloc[:, 2],49,0))
            sliding_average_cl = np.mean(signal.savgol_filter(self.history_force_Coeffs.iloc[:, 3],49,0))
        else:
            sliding_average_cd = np.mean(signal.savgol_filter(self.history_force_Coeffs.iloc[-sampling_num:, 2],49,0))
            sliding_average_cl = np.mean(signal.savgol_filter(self.history_force_Coeffs.iloc[-sampling_num:, 3],49,0))
        return sliding_average_cd, sliding_average_cl


    def sliding_history_force_coeffs(self, sliding_time_interval):
        sampling_num = int(sliding_time_interval / self.foam_params['delta_t'])
        if self.history_force_Coeffs.shape[0] <= sampling_num:
            sliding_history_cd = self.history_force_Coeffs.iloc[:, 2]
            sliding_history_cl = self.history_force_Coeffs.iloc[:, 3]
        else:
            sliding_history_cd = self.history_force_Coeffs.iloc[-sampling_num:, 2]
            sliding_history_cl = self.history_force_Coeffs.iloc[-sampling_num:, 3]
        return sliding_history_cd.to_numpy(), sliding_history_cl.to_numpy()

## Step 2: Define the parameters of the environment, DRL agent, and the training process.

Noted: `foam_params` needs to be defined according to the actual situation, especially for the `of_env_init` which aims to set proper OpenFOAM environment variables.

In [ ]:
import argparse
import re
from tensorforce import Runner, Agent, Environment
import os

# define parameters
number_servers=6
nstate=1
naction=1
foam_params = {
    'delta_t': 0.0005,
    'solver': 'pimpleFoam',
    'num_processor': 5,
    'of_env_init': 'source ~/OpenFOAM/OpenFOAM-8/etc/bashrc',
    'cfd_init_time': 0.005,
    'num_dimension': 2,
    'verbose': False
}

entry_dict_q0 = {
    'U': {
        'JET1': {
            'q0': '{x}',
        },
        'JET2': {
            'q0': '{-x}',
        }
    }
}

entry_dict_q1 = {
    'U': {
        'JET1': {
            'q1': '{y}',
        },
        'JET2': {
            'q1': '{-y}',
        }
    }
}

entry_dict_t0 = {
    'U': {
        'JET1': {
            't0': '{t}'
        },
        'JET2': {
            't0': '{t}'
        }
    }
}

agent_params = {
    'entry_dict_q0': entry_dict_q0,
    'entry_dict_q1': entry_dict_q1,
    'entry_dict_t0': entry_dict_t0,
    'deltaA': 0.05,
    'minmax_value': (-1.5, 1.5),
    'interaction_period': 0.025,
    'purgeWrite_numbers': 0,
    'writeInterval': 0.025,
    'deltaT': 0.0005,
    'variables_q0': ('x',),
    'variables_q1': ('y',),
    'variables_t0': ('t',),
    'verbose': False,
    "zero_net_Qs": True,
}
state_params = {
    'type': 'velocity'
}

# Pre-defined or custom environment
root_path = os.getcwd()
env_name_list = sorted([envs for envs in os.listdir(root_path) if re.search(r'^env\d+$', envs)])
environments = []
for env_name in env_name_list:
    env = FlowAroundCylinder2D(
        foam_root_path='/'.join([root_path, env_name]),
        foam_params=foam_params,
        agent_params=agent_params,
        state_params=state_params,
    )
    environments.append(env)

use_best_model = True
if use_best_model:
    evaluation_environment = environments.pop()
else:
    evaluation_environment = None

network_spec = [
    dict(type='dense', size=512,activation='tanh'),
    dict(type='dense', size=512,activation='tanh')
]
baseline_spec = [   
   dict(type='dense', size=512,activation='tanh'),
    dict(type='dense', size=512,activation='tanh')
]

# Train for 1000 episodes, each episode contain 100 actions
num_episodes = 1000
max_episode_timesteps = 100

# Instantiate a Tensorforce agent
agent = Agent.create(
    agent='ppo',
    environment=env,max_episode_timesteps=max_episode_timesteps,
    batch_size=20,
     network=network_spec,
    learning_rate=0.001,state_preprocessing=None,
    entropy_regularization=0.01, likelihood_ratio_clipping=0.2, subsampling_fraction=0.2,
    predict_terminal_values=True,
    discount=0.97,
    baseline=baseline_spec,
    baseline_optimizer=dict(
        type='multi_step',
        optimizer=dict(
            type='adam',
            learning_rate=1e-3
        ),
        num_steps=5
    ),
    multi_step=25,
    parallel_interactions=number_servers,
    saver=dict(directory=os.path.join(os.getcwd(), 'saved_models/checkpoint'),frequency=1  
    # save checkpoint every 100 updates
    ),
    summarizer=dict(
        directory='summary',
        # list of labels, or 'all'
        labels=['entropy', 'kl-divergence', 'loss', 'reward', 'update-norm']
    ),
)
# print(agent.get_architecture())
# agent = Agent.load(directory='saved_models/checkpoint')

print('Agent defined DONE!')


## Step 3: Start trainning the DRL agent.

In [ ]:
runner = Runner(
    agent=agent,
    environments=environments,
    max_episode_timesteps=max_episode_timesteps,
    evaluation=use_best_model,
    remote='multiprocessing',
)
print('Runner defined DONE!')

# runner.run(episodes=500, max_episode_timesteps=80)
runner.run(
    num_episodes=num_episodes,
    save_best_agent ='best_model'
)
runner.close()

for environment in environments:
    environment.close()
    